In [14]:
from devgpt_processing.load.model import Issue, Conversation
# sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
# pandas
import pandas as pd
# session
engine = create_engine('sqlite:///devgpt.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

# Get all title and body text from the issues table
issues = session.query(Issue).all()
issues_data = {'id': [issue.id for issue in issues], 'title': [issue.title for issue in issues], 'body': [issue.body for issue in issues], 'language': [issue.language for issue in issues]}
issues_df = pd.DataFrame(issues_data)

# Get all conversations
conversations = session.query(Conversation).all()
conversations_data = {'Prompt': [conversation.Prompt for conversation in conversations], 'Answer': [conversation.Answer for conversation in conversations], 'id': [conversation.id for conversation in conversations], 'language': [conversation.language for conversation in conversations]}
conversations_df = pd.DataFrame(conversations_data)

In [15]:
issues_df

,id,title,body,language
0,2289790261998994420,Add a copy button to the route field,https://chat.openai.com/share/cd39a2c9-720e-44...,en
1,4984332946441439969,Ability to transform proxied content,Right now this proxy isn't useful for anything...,en
2,2779930534008885561,Proxy Support,User should be able to use proxy at the collec...,en
3,-7767857181492296727,Read pom.xml,```\r\n<dependency>\r\n <groupId>org.apache...,en
4,7234362348087580839,Bayes Factor division,What are the marginal likelihoods that should ...,en
...,...,...,...,...
3074,3430835302818826414,"Could you please add Dockerfile, docker-compos...",Could you please add\r\n\r\n* [ ] Dockerfile\r...,en
3075,9018493530821605019,/dev/hidraw[x] unreliable and /dev/serial/by-i...,"Hi,\r\n\r\nI have the issue that my jablotron ...",en
3076,-6473084198156186771,Refactor events to use emitter pattern.,GPT helped define this.\r\n\r\nhttps://chat.op...,en
3077,-29034395518929098,Auto documentation generation,Create a script that checks the `master` branc...,en


In [45]:
lang_list = issues_df['language'].unique()
# change the language code to the language name
from langcodes import Language
lang_dict = {}
for lang in lang_list:
    lang_dict[lang] = Language.make(lang).language_name().lower()

lang_dict

{'en': 'english',
 'ko': 'korean',
 'ja': 'japanese',
 'nl': 'dutch',
 'pt': 'portuguese',
 'fr': 'french',
 'es': 'spanish',
 'zh': 'chinese',
 'de': 'german'}

In [48]:
from nltk.stem.snowball import SnowballStemmer
# list of supported languages from snowballstemmer
stemmer_list = SnowballStemmer.languages
stemmer_list


('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [49]:
from nltk.stem.snowball import SnowballStemmer
def get_stemmer(lang):
    return SnowballStemmer(lang)

def split_text(text):
    return text.split(' ')

# titles is a list of list [[[title words], language]]
titles = issues_df[['title', 'language']].apply(lambda x: [split_text(x[0]), x[1]], axis=1).tolist()

# stem the words
stemmed_titles = []
for title in titles:
    lang = lang_dict[title[1]]
    if lang not in stemmer_list:
        continue
    sentence = ' '.join([get_stemmer(lang).stem(word) for word in title[0]])
    stemmed_titles.append(sentence)



/var/folders/1s/d6cby_d90fsfb94qvy0p0ttm0000gn/T/ipykernel_8750/2480575227.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  titles = issues_df[['title', 'language']].apply(lambda x: [split_text(x[0]), x[1]], axis=1).tolist()


In [50]:
stemmed_titles

['add a copi button to the rout field',
 'abil to transform proxi content',
 'proxi support',
 'read pom.xml',
 'bay factor divis',
 'auto document generat',
 'error handl',
 'clean instal gives: npm err! notarget no match version found for docsify-cli@^4.4.7.',
 'generat unit test for util function with ai',
 'unknown field `eval`, expect one of `properties`, ...',
 '.gptignor should appropri interpret lead slash',
 'prevent duplic ',
 'fail to decod download font',
 'https://chat.openai.com/share/a3634550-9752-479e-937a-b3db80c88449',
 'no consol output on v0.7.0 (run on chatgpt code interpreter)',
 'featur request: add url paramet to allow share the sub-task view in the kanban on calls, chat, etc. [15.0]',
 'generat record attempt overrid an includ field with a record type that is not a subtyp of the origin type',
 'rollback fail to elimin directori and also fail to report it error correctly.',
 'how to support navig',
 'helper to aggreg total',
 'wrapper into independ runner os',
 